<a href="https://colab.research.google.com/github/stygian2a/RoombaGym/blob/master/Roomba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO


In [0]:
!pip uninstall keras -y   > /dev/null 2>&1
!pip install keras==2.2.4   > /dev/null 2>&1

!pip uninstall tensorflow -y   > /dev/null 2>&1
!pip install tensorflow==1.13.1  > /dev/null 2>&1

#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [12]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!pip install keras-rl   > /dev/null 2>&1

!wget https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)
--2020-02-29 11:05:47--  https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11819 (12K) [text/plain]
Saving to: ‘rendering.py.5’

rendering.py.5      100%[===================>]  11.54K  --.-KB/s    in 0s      

2020-02-29 11:05:47 (169 MB/s) - ‘rendering.py.5’ saved [11819/11819]



In [0]:
import gym
from gym import spaces
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from time import sleep
from scipy.stats import multivariate_normal

from IPython import display as ipythondisplay

In [14]:
!dpkg --configure -a
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.


In [15]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1013'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## Environment

In [0]:
class Roomba():
  def __init__(self, pos, direction, radius, battery, turn_angle=np.pi/9):
    self.pos = np.array(pos)
    self.direction = direction
    self.radius = radius
    self.turn_angle = turn_angle
    self.battery = battery
    self.max_battery = battery

  def set_pos(self, pos):
    self.pos = np.array(pos)

  def set_direction(self, direction):
    self.direction = np.array(direction)

  def get_forward(self, direction=None):
    if direction == None:
      direction = self.direction
    new_x = np.sin(direction) + self.pos[0]
    new_y = np.cos(direction) + self.pos[1]
    return [new_x, new_y]

  def get_backward(self, direction=None):
    if direction == None:
      direction = self.direction
    new_x = np.sin(direction+np.pi) + self.pos[0]
    new_y = np.cos(direction+np.pi) + self.pos[1]
    return [new_x, new_y]

  def turn_right(self):
    return (self.direction - self.turn_angle) % (2*np.pi)

  def turn_left(self):
    return (self.direction + self.turn_angle) % (2*np.pi)

  def decrease_battery(self):
    self.battery -=1

  def recharge_battery(self):
    self.battery = self.max_battery

  def get_life(self):
    return self.battery / self.max_battery


  #https://stackoverflow.com/questions/49551440/python-all-points-on-circle-given-radius-and-center
  def get_cases_underneath(self, xmax, ymax, pos = None):
    if pos == None:
      pos = self.pos
    radius = self.radius
    x0 = pos[0] - 0.5
    y0 = pos[1] - 0.5

    L = []
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    for x, y in zip(x_[x], y_[y]):
      if x>=0 and y>=0 and x<xmax and y<ymax:
        L.append((x, y))
    return L

In [0]:
class RoombaEnv(gym.Env):

  metadata = {
      'render.modes': ['human', 'rgb_array'],
      'video.frames_per_second' : 50
  }

  def __init__(self, height, width, battery, roomba_radius=2):
    super(RoombaEnv, self).__init__()

    self.action_space = spaces.Discrete(4) #Forward, Backwards, Right, Left

    """                                      
    self.observation_space = spaces.Tuple([
        spaces.Box(low=0, high=4, shape=(height, width), dtype=np.float32), #1=cleanable, 2=wall, 3=charging station
        spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    ])
    """
    #self.observation_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([1, 1, 2*np.pi, 1]), dtype=np.float32)
    self.observation_space = spaces.Box(low=np.array(np.zeros(4+height*width)), high=np.concatenate([np.array([1, 1, 2*np.pi, 1]), np.ones(height*width)]), dtype=np.float32)

    self.width = width
    self.height = height
    self.battery = battery
    self.roomba_radius = roomba_radius

    self.cases_indexes = np.array( [(i,j) for i in range(height) for j in range(width)] )
    self.dirt_probabilities = multivariate_normal.pdf(self.cases_indexes, mean=[height/2,width/2], cov=10*np.diag([1,1]))
    self.dirt_probabilities = self.dirt_probabilities / np.sum(self.dirt_probabilities)

    self.room = np.ones((height, width))
    #obstacles = [[5,5], [5,6], [5,7], [5,8], [6,5], [7,5]]
    obstacles = []
    for obstIndex in obstacles:
      self.room[obstIndex[0],obstIndex[1]]= 2
    for i in range(4):
      for j in range(4):
        self.room[i, j] = 3
    self.viewer = None
    self.init_env()

  def init_env(self):
    self.roomba = Roomba( [self.height-3, self.width-3], -np.pi/2, self.roomba_radius, self.battery)
    self.done = False
    self.iteration = 0
    self.sum_rewards = 0
    self.dirty = np.ones((self.height, self.width))


  def clean_floor(self):
    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width)
    reward = 0
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 1:
        reward += min(1,self.dirty[case_underneath])
        self.dirty[case_underneath] = 0

      if self.room[case_underneath] == 3:
        if self.roomba.get_life() < 0.5:
          reward += 100
        self.roomba.recharge_battery()
    return reward

  def soil_floor(self):
    i = np.random.choice(self.height * self.width, p=self.dirt_probabilities)
    self.dirty[self.cases_indexes[i][0], self.cases_indexes[i][1]] = 1

  def step(self, action):
    self.iteration+=1
    #if random.random() <0.1:
    #  action = random.randint(0,3)
    #current_reward = 1
    current_reward = 0
    if action == 0:
      new_pos = self.roomba.get_forward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 1:
      new_pos = self.roomba.get_backward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 2:
      self.roomba.set_direction(self.roomba.turn_right())

    elif action == 3:
      self.roomba.set_direction(self.roomba.turn_left())
    self.roomba.decrease_battery()
      
    current_reward += self.clean_floor()
    
    #cost of living
    #current_reward -= 1

    self.soil_floor()

    if self.roomba.battery <= 0:
      current_reward += -200

    self.sum_rewards += current_reward

    if self.iteration>=10000 or self.roomba.battery <= 0:
      self.done = True
      
    return self.get_observation(),current_reward,self.done,{}

  def is_safe(self, new_pos):
    x = new_pos[0]
    y = new_pos[1]

    if y < 0 or x < 0 or x >= self.height or y >= self.width:
      return False

    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width, new_pos)
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 2:
        return False

    return True

  def get_observation(self):
    #return np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()])
    return np.concatenate([np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()]), self.dirty.flatten()])

    #return (self.dirty, np.array([self.roomba.pos[0], self.roomba.pos[1], self.roomba.direction, self.roomba.get_life()]))

  def reset(self):
    self.init_env()
    return self.get_observation()

  def render(self, mode='human', close=False):
    scale = 6

    screen_width = scale*self.width
    screen_height = scale*self.height


    if self.viewer is None:
      #TODO: remove when it works
      import importlib
      import rendering
      importlib.reload(rendering)

      self.viewer = rendering.Viewer(screen_width, screen_height)

      self.cases = []
      for i in range(self.height):
        self.cases.append([])
        for j in range(self.width):
          x0 = i*scale
          y0 = j*scale
          x1 = (i+1)*scale
          y1 = (j+1)*scale
          case = rendering.FilledPolygon([(y0,x0), (y0,x1), (y1,x1), (y1,x0)])
          self.cases[i].append(case)
          self.viewer.add_geom(case)
      
      circle = rendering.make_circle(radius=self.roomba.radius*scale)
      circle.set_color(1,0,0)
      self.circle=circle
      direction_circle = rendering.make_circle(radius=self.roomba.radius*scale/2) #color?
      direction_circle.add_attr(rendering.Transform(translation=(self.roomba.radius*scale/2,0)))

      tracker = rendering.Compound([circle, direction_circle])

      self.tracker_trans = rendering.Transform()
      tracker.add_attr(self.tracker_trans)
      self.viewer.add_geom(tracker)

      
    for i in range(self.height):
        for j in range(self.width):
          if self.room[i,j] == 0:
            self.cases[i][j].set_color(1,1,1)
          elif self.room[i,j] == 1:
            self.cases[i][j].set_color(1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]))
          elif self.room[i,j] == 2:
            self.cases[i][j].set_color(0,0,0)
          elif self.room[i,j] == 3:
            self.cases[i][j].set_color(0.5,0.5,0)


    self.tracker_trans.set_translation(self.roomba.pos[1]*scale, self.roomba.pos[0]*scale)
    self.tracker_trans.set_rotation(self.roomba.direction)
    self.circle.set_color(self.roomba.get_life(),0,0)


    return self.viewer.render(return_rgb_array = mode=='rgb_array')

In [19]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done:
      break;
            
env.close()
show_video()

## Baseline

In [0]:
#TODO : Think of BETTER FEATURES (better value for turning)

Height = 20
Width = 20
battery = 500
pad = 2

env = wrap_env(RoombaEnv(Height, Width, battery, pad))
possible_actions = [env.roomba.get_forward, env.roomba.get_backward, env.roomba.turn_right, env.roomba.turn_left]

def closestDirt(x, y, mat):
  d = Height+Width
  for k in range(Height): 
    for l in range(Width):  
        if (mat[k][l] == 1): 
            d = min(d, abs(x - k) + abs(y - l))
  return d

def feature1(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  x_min = max(0,int(pos[0])-pad)
  x_max = min(Height,int(pos[0])+pad)
  y_min = max(0,int(pos[1])-pad)
  y_max = min(Width,int(pos[1])+pad)
  AAA = np.sum(dirtMap[x_min:x_max, y_min:y_max])/(pad*pad)
  AAA -= turn/(pad*pad)
  return AAA

def feature2(state, action):
  charger = np.array([Height-3,Width-3])
  life = state[3]
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  else:
    life -= - 1/500
  distManhattan = charger-pos
  distManhattan = np.sum(np.abs(distManhattan))
  #print(min((1-life)/distManhattan,1))
  return min((1-life)/distManhattan,1)

def feature3(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  BBB = closestDirt(int(pos[0]),int(pos[1]), dirtMap)/(Height+Width)
  BBB -= turn/(Height+Width)
  return BBB

def feature4(state, action):
  return 1

In [0]:
# Hyperparameters
alpha = 0.1
gamma = 0.9999
epsilon = 0.3

weights = [0, 0, 0, 0]
F = [feature1, feature2, feature3, feature4]

def getQValue(state, action):
  qv = 0
  for i, feature in enumerate(F):
      qv = qv + weights[i] * feature(state, action)
  return qv

for e in range(5):
  done = False
  state = env.reset()
  while not done:
    if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
    else:
        q = np.array([getQValue(state, a) for a in possible_actions])
        action = np.argmax(q)

    next_state, reward, done, info = env.step(action)

    action = possible_actions[action]

    next_values = [getQValue(next_state, a) for a in possible_actions]
    next_max = max(next_values)

    alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
    
    for i, feature in enumerate(F):
        weights[i] = weights[i] + alphadiff * feature(state, action)

    state = next_state

state = env.reset()
while not done:
  env.render()
  q = np.array([getQValue(state, a) for a in possible_actions])
  action = np.argmax(q)

  next_state, reward, done, info = env.step(action)

  action = possible_actions[action]

  next_values = [getQValue(next_state, a) for a in possible_actions]
  next_max = max(next_values)

  alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
  
  for i, feature in enumerate(F):
      weights[i] = weights[i] + alphadiff * feature(state, action)

  state = next_state

env.close()
show_video()

In [0]:
weights

[22967759054.38774, 32837938137.780552, 2237315975.4568067, 51688288572.63189]

## DQN - Big


In [0]:

import numpy as np
import gym

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, Reshape, Lambda
from keras.models import Model

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, GreedyQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

from keras.layers import Dense, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, Reshape, Lambda
from keras.models import Model

env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse

nb_actions = env.action_space.n


# Next, we build a very simple model.

i = Input(shape=(1,)+env.observation_space.shape)

#input_roomba, input_map = tf.split(i, [4, env.width*env.height], 1)
input_roomba = Lambda(lambda x: x[:,:,0:4])(i)
input_roomba = Flatten()(input_roomba)

input_map = Lambda( lambda x: x[:,:,4:])(i)


map_reshaped = Reshape((env.height, env.width,1))(input_map)

conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(map_reshaped)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Flatten()(conv)
conv = Dense(64)(conv)
conv = Dense(32)(conv)
conv = Dense(16)(conv)

merged = Concatenate()([conv, input_roomba])

merged = Dense(8)(merged)
output = Dense(nb_actions)(merged)

model = Model(inputs=i, outputs=output)
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=60000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=4000, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
dqn.save_weights('dqn_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=False)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 1, 404)       0                                            
__________________________________________________________________________________________________
lambda_44 (Lambda)              (None, 1, 400)       0           input_22[0][0]                   
__________________________________________________________________________________________________
reshape_22 (Reshape)            (None, 20, 20, 1)    0           lambda_44[0][0]                  
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 18, 18, 16)   160         reshape_22[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [0]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))

dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

In [0]:
! rm -rf video/

env = wrap_env(RoombaEnv(20, 20, 500, 2))

dqn.test(env, nb_episodes=5, visualize=True)
            
env.close()
show_video()

Testing for 5 episodes ...
Episode 1: reward: 2.000, steps: 10000
Episode 2: reward: 212.000, steps: 2777
Episode 3: reward: -141.000, steps: 609
Episode 4: reward: 11.000, steps: 10000
Episode 5: reward: -67.000, steps: 1030


In [0]:
! rm -rf video/

env = wrap_env(RoombaEnv(20, 20, 500, 2))

dqn.fit(env, nb_steps=2000, visualize=True, verbose=2)
            
env.close()
show_video()

Training for 2000 steps ...
  615/2000: episode: 1, duration: 10.692s, episode steps: 615, steps per second: 58, episode reward: -95.000, mean reward: -0.154 [-200.000, 4.000], mean action: 1.485 [0.000, 3.000], mean observation: 0.237 [0.000, 6.109], loss: --, mean_absolute_error: --, mean_q: --
 1309/2000: episode: 2, duration: 11.726s, episode steps: 694, steps per second: 59, episode reward: -192.000, mean reward: -0.277 [-200.000, 3.000], mean action: 1.899 [0.000, 3.000], mean observation: 0.325 [-1.571, 6.109], loss: --, mean_absolute_error: --, mean_q: --
done, took 28.280 seconds


## DQN - Small


In [0]:

import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, GreedyQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory



env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse

nb_actions = env.action_space.n


# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=60000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

# After training is done, we save the final weights.
dqn.save_weights('dqn_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=False)


Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 404)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                6480      
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

# CEM

In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory


from keras.layers import Dense, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, Reshape, Lambda
from keras.models import Model

env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse

nb_actions = env.action_space.n


# Next, we build a very simple model.

i = Input(shape=(1,)+env.observation_space.shape)

#input_roomba, input_map = tf.split(i, [4, env.width*env.height], 1)
input_roomba = Lambda(lambda x: x[:,:,0:4])(i)
input_roomba = Flatten()(input_roomba)

input_map = Lambda( lambda x: x[:,:,4:])(i)


map_reshaped = Reshape((env.height, env.width,1))(input_map)

conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(map_reshaped)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Flatten()(conv)
conv = Dense(64)(conv)
conv = Dense(32)(conv)
conv = Dense(16)(conv)

merged = Concatenate()([conv, input_roomba])

merged = Dense(8)(merged)
output = Dense(nb_actions)(merged)

model = Model(inputs=i, outputs=output)


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=100000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=1000, nb_steps_warmup=1000, train_interval=500, elite_frac=0.05)

#cem.load_weights('cem_roomba_params.h5f')
cem.compile()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 1, 404)       0                                            
__________________________________________________________________________________________________
lambda_34 (Lambda)              (None, 1, 400)       0           input_17[0][0]                   
__________________________________________________________________________________________________
reshape_17 (Reshape)            (None, 20, 20, 1)    0           lambda_34[0][0]                  
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 18, 18, 16)   160         reshape_17[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [0]:
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=200000, visualize=False, verbose=1)

# After training is done, we save the best weights.
cem.save_weights('cem_roomba_params.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=False)

Training for 200000 steps ...
Interval 1 (0 steps performed)
  994/10000 [=>............................] - ETA: 12s - reward: 0.0121

/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 11s 1ms/step - reward: -0.1114
9 episodes - episode_reward: -131.778 [-199.000, 84.000] - mean_best_reward: -188.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 10s 1000us/step - reward: -0.1517
12 episodes - episode_reward: -137.917 [-195.000, -26.000]

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 10s 1ms/step - reward: -0.1898
13 episodes - episode_reward: -139.846 [-200.000, 62.000]

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 10s 1ms/step - reward: -0.0523
7 episodes - episode_reward: -57.143 [-200.000, 205.000]

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 11s 1ms/step - reward: -0.1577
11 episodes - episode_reward: -142.727 [-198.000, -53.000]

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 10s 1ms/step - reward: -0.1374
10 episodes - episode_reward: 

In [0]:
! rm -rf video/

env = wrap_env(RoombaEnv(20, 20, 500, 2))

cem.fit(env, nb_steps=10000, visualize=True, verbose=2)
            
env.close()
show_video()

Training for 10000 steps ...


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 8672/10000: episode: 1, duration: 141.505s, episode steps: 8672, steps per second: 61, episode reward: 1894.000, mean reward: 0.218 [-200.000, 100.000], mean action: 0.813 [0.000, 3.000], mean observation: 0.439 [0.000, 6.109], mean_best_reward: 1894.000000
done, took 163.095 seconds


In [0]:
random.random()

# Sarsa - Big

In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
i = Input(shape=(1,)+env.observation_space.shape)

#input_roomba, input_map = tf.split(i, [4, env.width*env.height], 1)
input_roomba = Lambda(lambda x: x[:,:,0:4])(i)
input_roomba = Flatten()(input_roomba)

input_map = Lambda( lambda x: x[:,:,4:])(i)


map_reshaped = Reshape((env.height, env.width,1))(input_map)

conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(map_reshaped)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(16, kernel_size=(3, 3),
                 activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Flatten()(conv)
conv = Dense(64)(conv)
conv = Dense(32)(conv)
conv = Dense(16)(conv)

merged = Concatenate()([conv, input_roomba])

merged = Dense(8)(merged)
output = Dense(nb_actions)(merged)

model = Model(inputs=i, outputs=output)
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10000, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 404)       0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 1, 400)       0           input_3[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 20, 20, 1)    0           lambda_6[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 18, 18, 16)   160         reshape_3[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [0]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

sarsa.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

Testing for 1 episodes ...
Episode 1: reward: -10000.000, steps: 10000


# Sarsa - Small


In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10000, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_9 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_10 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 68        
_________________________________________________________________
activation_11 (Activation)   (None, 4)                 0         
Total para

# Stable Baselines Lib


In [20]:
!pip install stable-baselines

## PPO2

In [44]:
#check how to display
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2


env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)
env.close()

Streaming output truncated to the last 5000 lines.
| policy_entropy     | 1.3537579     |
| policy_loss        | 0.0018270061  |
| serial_timesteps   | 768           |
| time_elapsed       | 7.8           |
| total_timesteps    | 768           |
| value_loss         | 35.886127     |
--------------------------------------
-------------------------------------
| approxkl           | 0.0006866022 |
| clipfrac           | 0.0          |
| explained_variance | 0.0261       |
| fps                | 85           |
| n_updates          | 7            |
| policy_entropy     | 1.3651757    |
| policy_loss        | -0.002362391 |
| serial_timesteps   | 896          |
| time_elapsed       | 9.26         |
| total_timesteps    | 896          |
| value_loss         | 2.3887925    |
-------------------------------------
--------------------------------------
| approxkl           | 0.000930146   |
| clipfrac           | 0.0           |
| explained_variance | 0.00091       |
| fps                | 101

In [45]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
for i in range(1000):
    env.render()
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
      break
env.close()
show_video()

## DQN

In [32]:
import gym

from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import DQN

env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = DQN(MlpPolicy, env,exploration_fraction=0.2, exploration_final_eps=0.02, verbose=1)
model.learn(total_timesteps=100000)
model.save("EE")

#del model # remove to demonstrate saving and loading
# model = DQN.load("EE")

env.close()

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 100      |
| mean 100 episode reward | 75.9     |
| steps                   | 77330    |
--------------------------------------


In [0]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()

while True:
    env.render()
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
      break
env.close()
show_video()

## TRPO

In [34]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines import TRPO

env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = TRPO(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=100000)
model.save("EE2")

env.close()

********** Iteration 0 ************
Optimizing Policy...
sampling
done in 11.702 seconds
computegrad
done in 0.132 seconds
conjugate_gradient
      iter residual norm  soln norm
         0     0.0284          0
         1    0.00206     0.0332
         2    0.00106      0.364
         3    0.00345      0.519
         4   0.000868      0.548
         5   0.000235      0.928
         6    0.00428      0.997
         7   9.15e-05       1.11
         8   5.13e-05       1.26
         9   7.85e-06       1.27
        10   6.54e-05       1.27
done in 0.211 seconds
Expected: 0.029 Actual: 0.027
Stepsize OK!
vf
done in 0.132 seconds
------------------------------------------
| EpLenMean               | 500          |
| EpRewMean               | -57          |
| EpThisIter              | 2            |
| EpisodesSoFar           | 2            |
| TimeElapsed             | 12.3         |
| TimestepsSoFar          | 1024         |
| entloss                 | 0.0          |
| entropy                

In [35]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
      break
env.close()
show_video()

## A2C

In [36]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C

env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)
model.save("EE3")

Wrapping the env in a DummyVecEnv.
---------------------------------
| explained_variance | -0.0414  |
| fps                | 20       |
| nupdates           | 1        |
| policy_entropy     | 1.39     |
| total_timesteps    | 5        |
| value_loss         | 56.4     |
---------------------------------
---------------------------------
| explained_variance | -0.0107  |
| fps                | 80       |
| nupdates           | 100      |
| policy_entropy     | 1.39     |
| total_timesteps    | 500      |
| value_loss         | 3.81e+04 |
---------------------------------
---------------------------------
| explained_variance | -0.0145  |
| fps                | 82       |
| nupdates           | 200      |
| policy_entropy     | 1.39     |
| total_timesteps    | 1000     |
| value_loss         | 3.81e+04 |
---------------------------------
---------------------------------
| explained_variance | 0.0186   |
| fps                | 117      |
| nupdates           | 300      |
| policy_entr

In [37]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
      break
env.close()
show_video()

## GAIL

In [49]:
import gym

from stable_baselines import GAIL, SAC
from stable_baselines.gail import ExpertDataset, generate_expert_traj

# Generate expert trajectories (train expert)
env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = A2C(MlpPolicy, env, verbose=1)
generate_expert_traj(model, 'expert_roomba', n_timesteps=2000, n_episodes=10)

# Load the expert dataset
dataset = ExpertDataset(expert_path='expert_roomba.npz', traj_limitation=100, verbose=1)

model = GAIL('MlpPolicy', env, dataset, verbose=1)
# Note: in practice, you need to train for 1M steps to have a working policy
model.learn(total_timesteps=50000)
model.save("gail_roomba")

env.close()

Wrapping the env in a DummyVecEnv.


Exception ignored in: <bound method ExpertDataset.__del__ of <stable_baselines.gail.dataset.dataset.ExpertDataset object at 0x7fa53a48a390>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/stable_baselines/gail/dataset/dataset.py", line 121, in __del__
    del self.dataloader, self.train_loader, self.val_loader
AttributeError: dataloader


---------------------------------
| explained_variance | -0.0264  |
| fps                | 19       |
| nupdates           | 1        |
| policy_entropy     | 1.39     |
| total_timesteps    | 5        |
| value_loss         | 58.4     |
---------------------------------
---------------------------------
| explained_variance | 0.00437  |
| fps                | 80       |
| nupdates           | 100      |
| policy_entropy     | 1.39     |
| total_timesteps    | 500      |
| value_loss         | 3.85e+04 |
---------------------------------
---------------------------------
| explained_variance | -0.00215 |
| fps                | 83       |
| nupdates           | 200      |
| policy_entropy     | 1.39     |
| total_timesteps    | 1000     |
| value_loss         | 3.71e+04 |
---------------------------------
---------------------------------
| explained_variance | -0.00279 |
| fps                | 118      |
| nupdates           | 300      |
| policy_entropy     | 1.39     |
| total_timest

In [50]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
      break
env.close()
show_video()

## ACER

In [39]:
import gym

from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import ACER

env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = ACER(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)
model.save("EE4")

Wrapping the env in a DummyVecEnv.
----------------------------------
| avg_norm_adj        | 0        |
| avg_norm_g          | 34.5     |
| avg_norm_grads_f    | 34.5     |
| avg_norm_k          | 2        |
| avg_norm_k_dot_g    | 34.5     |
| entropy             | 29.1     |
| explained_variance  | 0.000768 |
| fps                 | 0        |
| loss                | 35.7     |
| loss_bc             | -0       |
| loss_f              | 11.9     |
| loss_policy         | 11.9     |
| loss_q              | 48       |
| mean_episode_length | 0        |
| mean_episode_reward | 0        |
| norm_grads          | 21.3     |
| norm_grads_policy   | 16.1     |
| norm_grads_q        | 13.9     |
| total_timesteps     | 0        |
----------------------------------
----------------------------------
| avg_norm_adj        | 0        |
| avg_norm_g          | 5.52     |
| avg_norm_grads_f    | 5.52     |
| avg_norm_k          | 2.12     |
| avg_norm_k_dot_g    | 5.46     |
| entropy           

In [40]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
      break
env.close()
show_video()

## ACKTR

In [41]:
import gym

from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import ACKTR

env = wrap_env(RoombaEnv(20, 20, 500, 2))

model = ACKTR(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=50000)
model.save("E5")

Wrapping the env in a DummyVecEnv.
---------------------------------
| explained_variance | 0.00369  |
| fps                | 21       |
| nupdates           | 1        |
| policy_entropy     | 1.39     |
| policy_loss        | 12       |
| total_timesteps    | 0        |
| value_loss         | 137      |
---------------------------------
---------------------------------
| explained_variance | 0.0237   |
| fps                | 68       |
| nupdates           | 100      |
| policy_entropy     | 1.15     |
| policy_loss        | 10.1     |
| total_timesteps    | 2079     |
| value_loss         | 214      |
---------------------------------
---------------------------------
| explained_variance | -0.0084  |
| fps                | 101      |
| nupdates           | 200      |
| policy_entropy     | 0.642    |
| policy_loss        | -0.232   |
| total_timesteps    | 4179     |
| value_loss         | 0.213    |
---------------------------------
---------------------------------
| explained_v

In [42]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
      break
env.close()
show_video()

In [0]:
#TODO replace by this the repetitions!
def asses(model):
  env = wrap_env(RoombaEnv(20, 20, 500, 2))
  obs = env.reset()
  while True:
      action, _states = model.predict(obs)
      obs, rewards, done, info = env.step(action)
      env.render()
      if done:
        break
  env.close()
  show_video()